In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import re
from collections import Counter

In [2]:
# Change the Matplotlib defaults
colors = np.array([(182,109,255),(218,109,0),(0,146,146)])/255
plt.rcParams.update({'font.size': 16, 'figure.figsize': (12.0, 6.0)})
plt.style.use('seaborn')
%matplotlib inline

# Load the song lyrics

In [41]:
fp = "./data/SongLyrics/genius_lyrics_subset.json"
songs = pd.read_json(fp)
songs.head(2)

,title,album,year,lyrics,image,artist,raw,gender,genre
42110,Rocket,Siamese Dream,1994-07-01,Bleed in your own light\nDream of your own lif...,https://images.genius.com/98ae047016f0e171782e...,Smashing Pumpkins,"{'annotation_count': 6, 'api_path': '/songs/13...",male,rock
53290,When Somebody Knows You That Well,Blake Shelton's Barn & Grill,None,"Tommy's daddy, kept a six pack in the fridge\n...",https://images.genius.com/e06a13f05b598aff9c84...,Blake Shelton,{'embed_content': '<div id='rg_embed_link_1854...,male,country


# Collect the Genius artist image URLs

In [123]:
image_urls = {}
columns = ["genre", "gender", "artist", "raw"]
for _, row in songs[columns].iterrows():
    genre = row['genre'].upper()
    gender = row['gender'].upper()
    ID = row['raw']['id']
    artist = row['artist']
    key = f"{genre}_{gender}_{ID}_{artist}".replace(" ", "_")
    image_urls[key] = row["raw"]["primary_artist"]["image_url"]

## Where are the images hosted?

In [38]:
from collections import Counter

In [39]:
domains = Counter(map(lambda x: x.split(".com")[0].replace("https://", ""), image_urls))
domains

Counter({'images.genius': 868,
         's3.amazonaws': 127,
         'i.genius': 1,
         'assets.genius': 3,
         'images.rapgenius': 1})

# Download and save the images

Or would it be better to not download them, just accessing each URL as I do the face detection?

In [89]:
import requests
import time

In [125]:
t0 = time.time()
prefix = "./data/images/"
for n, (genre_artist, url) in enumerate(image_urls.items()):
    # Format the filename
    genre, artist = genre_artist.split("_", 1)
    ext = url.rsplit(".")[-1]
    fp = prefix + genre_artist + "." + ext
    if n % 10 == 0:
        print(f"({n + 1:3.0f}/{len(image_urls)}): {fp}")

    # Download the image
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        image = r.raw.read()
        open(fp, "wb").write(image)
    else:
        print(f"(n={n})\nBad status code: {r.status_code}")
        print(f"Time elapsed: {(time.time() - t0) / 60:.2f} minutes.")
        time.sleep(3 * 60) # Wait three minutes
    time.sleep(2) # Wait two seconds
print(f"Total time elapsed: {(time.time() - t0) / 60:.2f} minutes.")

(  1/1000): ./data/images/ROCK_MALE_131735_Smashing_Pumpkins.jpg
( 11/1000): ./data/images/ROCK_MALE_399664_Fleetwood_Mac.jpg


KeyboardInterrupt: 